<a href="https://colab.research.google.com/github/99doyoon/forcapston/blob/main/firstClear_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import networkx as nx
import math
import numpy as np
import cv2
from sklearn.metrics.pairwise import euclidean_distances
from skimage.measure import approximate_polygon

`cv2.findContours` 함수는 이미지에서 컨투어(윤곽선)을 찾는 함수입니다. 이 함수는 바이너리 이미지를 입력으로 받아, 이미지의 픽셀 값이 0이 아닌 픽셀들로 이루어진 연속된 선(윤곽선)을 찾아냅니다. 이 함수는 주로 객체의 외곽선이나 이미지에서 관심 영역을 찾는데 사용됩니다.

함수의 인자는 다음과 같습니다:

- 첫 번째 인자는 바이너리 이미지입니다. 이 이미지에서 픽셀 값이 0이 아닌 부분이 컨투어로 간주됩니다.
- 두 번째 인자는 컨투어 추출 모드입니다. `cv2.RETR_EXTERNAL`을 사용하면 이미지의 가장 바깥쪽 컨투어만 찾아냅니다.
- 세 번째 인자는 컨투어 근사 방식입니다. `cv2.CHAIN_APPROX_SIMPLE`을 사용하면 컨투어의 수평, 수직, 대각선 방향의 세그먼트를 압축하여 끝점만 남깁니다. 즉, 불필요한 정보를 제거하여 메모리를 절약합니다.

함수의 반환값은 다음과 같습니다:

- 첫 번째 반환값은 찾아낸 컨투어입니다. 각 컨투어는 컨투어를 구성하는 점들의 리스트로 표현됩니다.
- 두 번째 반환값은 컨투어 계층 정보입니다. 이 정보는 이미지에 여러 계층의 컨투어가 있는 경우 각 컨투어가 어떤 계층에 속하는지를 나타냅니다. `cv2.RETR_EXTERNAL` 모드를 사용하면 가장 바깥쪽 컨투어만 찾으므로 이 정보는 필요 없습니다.

위 코드에서는 마스크 이미지에서 `cv2.findContours` 함수를 사용하여 컨투어를 찾고 있습니다. 마스크 이미지는 빨간색 부분만 픽셀 값이 0이 아니고 나머지 부분은 모두 0인 이미지이므로, 이 함수를 사용하면 빨간색 부분의 외곽선을 찾을 수 있습니다.

In [ ]:
!mkdir /content/rclone_drive
!rclone --config rclone.conf mount mydrive: /content/rclone_drive &

mkdir: cannot create directory ‘/content/rclone_drive’: File exists
/bin/bash: line 1: rclone: command not found


In [ ]:
!cat /root/.config/rclone/rclone.conf

cat: /root/.config/rclone/rclone.conf: No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/rclone.conf ~/.config/rclone/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/drive/MyDrive/rclone.conf': No such file or directory


In [ ]:
# 이미지 읽기
img = cv2.imread('/content/팀원지도최종본_방향추가.png')

# BGR에서 HSV로 변환
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 빨간색 범위를 정의
lower_red1 = np.array([0,50,50])
upper_red1 = np.array([10,255,255])
lower_red2 = np.array([170,50,50])
upper_red2 = np.array([180,255,255])

# HSV 이미지에서 빨간색만 추출하기 위한 임계값
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
mask = cv2.bitwise_or(mask1, mask2)

# 빨간색 마스크 이미지에서 컨투어 찾기
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

first_degree = None  # 첫 번째 각도 초기화
path_process = []  # 경로를 따라가는 과정 저장

for contour in contours:
    # 빨간색 객체의 면적이 너무 작으면 무시
    if cv2.contourArea(contour) < 500:
        continue

    # 빨간색 객체의 방향 찾기
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = 0, 0

    # 방향을 화살표로 그리기
    length = 100
    endX = cX + length * np.cos(M["mu11"]/M["m00"])
    endY = cY + length * np.sin(M["mu11"]/M["m00"])
    cv2.arrowedLine(img, (cX, cY), (int(endX), int(endY)), (255,0,0), 5)

    # 각도 계산
    dx = endX - cX
    dy = endY - cY
    angle_in_radians = math.atan2(dy, dx)
    angle_in_degrees = math.degrees(angle_in_radians)

    # 첫 번째 빨간색 객체의 각도를 저장
    if first_degree is None:
        first_degree = angle_in_degrees

    # 화살표가 바라보는 방향을 0도로 맞추기 위한 각도 조정
    adjusted_angle = 0 - first_degree
    path_process.append((cX, cY, adjusted_angle))

# 저장된 경로 과정 출력
for info in path_process:
    print(f"Position: {info[0]}, {info[1]}, Adjusted Angle: {info[2]}")

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
# 파란색 범위를 정의
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

# HSV 이미지에서 파란색만 추출하기 위한 임계값
blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

# 파란색 마스크 이미지에서 컨투어 찾기
blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

end = None  # 끝점 초기화

for contour in blue_contours:
    # 파란색 객체의 면적이 너무 작으면 무시
    if cv2.contourArea(contour) < 500:
        continue

    # 파란색 객체의 중심 찾기
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = 0, 0

    # 첫 번째 파란색 객체의 중심점을 끝점으로 저장
    if end is None:
        end = (cX, cY)

# 저장된 끝점 출력
print(f"End: {end}")

In [ ]:
# 빨간색과 파란색 마스크 생성
red_mask = cv2.inRange(hsv, lower_red1, upper_red1) | cv2.inRange(hsv, lower_red2, upper_red2)
blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

# 빨간색 또는 파란색인 부분을 하얀색으로 변경
img[(red_mask != 0) | (blue_mask != 0)] = [255, 255, 255]

# 이미지 저장
cv2.imwrite('/content/팀원지도최종본.png', img)

In [ ]:
# 이미지 불러오기
img = Image.open('/content/팀원지도최종본.png')

# 이미지를 흑백으로 변환
img = img.convert('L')

# 이미지의 픽셀 값을 가져오기
pixels = img.load()


In [ ]:
# 임계값 설정
threshold = 128

# 이진화 수행
for i in range(img.width):
    for j in range(img.height):
        if pixels[i, j] < threshold:
            pixels[i, j] = 0  # 통과 불가능한 지역
        else:
            pixels[i, j] = 1  # 통과 가능한 지역

In [ ]:
# 그래프 생성
G = nx.Graph()

# 노드 및 간선 추가
for i in range(img.width):
    for j in range(img.height):
        if pixels[i, j] == 1:  # 통과 가능한 지역만 노드로 추가
            G.add_node((i, j))

# 4방향 연결 고려
for node in G.nodes:
    x, y = node
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        node_x, node_y = x + dx, y + dy
        if (node_x, node_y) in G.nodes:  # 연결 가능한 노드만 간선으로 추가
            G.add_edge((x, y), (node_x, node_y))

nx.dijkstra_path(G, start, end)는 네트워크X 라이브러리의 일부로 Dijkstra 알고리즘을 구현한 함수입니다. 이 함수는 그래프 G에서 시작점 start와 종점 end 사이의 최단 경로를 찾는 역할을 합니다.

Dijkstra 알고리즘은 네트워크(그래프)에서 두 노드 사이의 최단 경로를 찾을 때 사용하는 알고리즘입니다. 이 알고리즘은 시작 노드에서 모든 다른 노드까지의 거리를 무한대로 설정한 후, 시작 노드의 거리를 0으로 설정하고 시작합니다.

다음으로, 알고리즘이 아직 방문하지 않은 노드 중에서 가장 거리가 짧은 노드를 선택합니다. 처음에는 시작 노드가 선택됩니다. 그 다음, 선택된 노드의 이웃 노드들의 거리를 업데이트합니다. 즉, 선택된 노드를 통해 이웃 노드에 도달하는 거리가 현재 저장된 거리보다 짧으면, 이웃 노드의 거리를 선택된 노드를 통해 도달하는 거리로 업데이트합니다.

이 과정을 모든 노드를 방문할 때까지 반복하면, 시작 노드에서 모든 노드까지의 최단 거리가 계산됩니다. 이 최단 거리 정보를 바탕으로 시작 노드에서 종점 노드까지의 최단 경로를 찾을 수 있습니다.

nx.dijkstra_path(G, start, end) 함수는 이 Dijkstra 알고리즘을 사용하여 시작점 start에서 종점 end까지의 최단 경로를 찾고, 이 경로를 노드의 리스트로 반환합니다. 이 리스트는 시작 노드에서 종점 노드까지의 최단 경로상의 노드들을 순서대로 담고 있습니다.

In [ ]:
# Dijkstra 알고리즘을 이용한 최단 경로 찾기
# 세로가 x(첫번째) 가로 y(두번째)
start, end = (100, 500), (500, 400)  # 시작점과 끝점 설정
path = nx.dijkstra_path(G, start, end)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# 원본 이미지 불러오기
img = Image.open('/content/팀원지도최종본.png')

# 이미지 출력 준비
fig, ax = plt.subplots()

# 원본 이미지 출력
ax.imshow(img, cmap='gray')

# colormap 지정 (점점 진해지는 파란색 계열)
cmap = plt.get_cmap('Blues')

# path에 저장된 노드들을 원으로 그리기
for i, node in enumerate(path):
    circle = plt.Circle((node[1], node[0]), radius=5, color=cmap(i / len(path)), fill=True)
    ax.add_patch(circle)

# 그래프 출력
plt.show()

Douglas-Peucker 알고리즘은 주로 2차원 평면에서 곡선이나 다각형의 꼭짓점을 줄여 단순화하는 데 사용되는 알고리즘입니다. 이 알고리즘은 곡선이나 다각형의 복잡도를 줄이면서도 전체적인 형태를 유지하는 데 효과적입니다. GPS 데이터와 같이 굉장히 많은 양의 데이터를 처리할 때 유용합니다.

Douglas-Peucker 알고리즘의 기본 원리는 다음과 같습니다:

1. 선분의 양 끝 점을 연결합니다. 이 선분을 기준선(Baseline)이라고 합니다.

2. 기준선에서 가장 멀리 떨어진 점을 찾습니다. 이 점을 가장 먼 점(Farthest Point)라고 합니다. 이 때의 거리를 오차 거리(Error Distance)라고 합니다.

3. 만약 가장 먼 점의 오차 거리가 미리 정한 오차 허용치(Tolerance)보다 작거나 같다면, 기준선을 최종 선분으로 사용합니다.

4. 오차 거리가 오차 허용치보다 크다면, 기준선을 두 개의 선분으로 나눕니다. 이 때, 한 선분의 끝점은 기준선의 한 끝점과 가장 먼 점이고, 다른 선분의 끝점은 가장 먼 점과 기준선의 다른 끝점입니다.

5. 나눈 선분에 대해 1~4의 과정을 재귀적으로 반복합니다. 이 때, 각 선분에 대해 오차 허용치를 초과하는 가장 먼 점이 없을 때까지 반복합니다.

6. 모든 선분에 대해 오차 허용치를 초과하는 가장 먼 점이 없으면 알고리즘을 종료하고, 남은 점들을 반환합니다.

이러한 방식으로 Douglas-Peucker 알고리즘이 작동하며, 원래의 곡선이나 다각형을 최대한 보존하면서도 점의 수를 크게 줄일 수 있습니다. 이 알고리즘은 곡선이나 다각형의 단순화뿐만 아니라 데이터 압축, 노이즈 제거 등 다양한 분야에서 활용됩니다.

In [ ]:
# 좌표를 NumPy 배열로 변환
coords = np.array(path)

# Douglas-Peucker 알고리즘 적용
tolerance = 1.0  # 허용 오차 설정
simplified_coords = approximate_polygon(coords, tolerance)

# 결과 출력
print(f"Original number of points: {len(coords)}")
print(f"Simplified number of points: {len(simplified_coords)}")

In [ ]:
# 경로의 각 좌표 사이의 거리와 방향 계산
for i in range(len(simplified_coords) - 1):
    vector = np.array(simplified_coords[i+1]) - np.array(simplified_coords[i])  # 현재 좌표와 다음 좌표를 연결하는 벡터
    direction = np.arctan2(vector[1], vector[0])  # 벡터의 방향
    distance = np.linalg.norm(vector)  # 벡터의 길이 (거리)

    # 경로를 따라가는 과정 저장
    path_process.append((distance, np.degrees(direction)))

# 결과 전송
for distance, direction in path_process:
    # 거리와 방향 정보를 문자열로 변환
    data = f"{distance},{direction}"

    # 아두이노에 데이터 전송
    ser.write(data.encode())

    # 아두이노가 데이터를 처리할 시간을 주기 위해 잠시 대기
    time.sleep(1)

# 연결 종료
ser.close()